https://github.com/rickiepark/handson-ml2/blob/master/12_custom_models_and_training_with_tensorflow.ipynb

설정

In [2]:
# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version은 코랩 명령입니다.
    %tensorflow_version 2.x
except Exception:
    pass

# 텐서플로 ≥2.0 필수
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)
tf.random.set_seed(42)

# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

##텐서와 연산

###텐서

In [6]:
xa=tf.constant([[1.,2,3], [4.,5,6]])
print(xa)

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)


numpy는 기본으로 64비트 정밀도를 사용하지만

tensorflow는 32비트.

넘파이 배열로 텐서를 만들때,
dtype=tf.float32로 지정해야 함

In [7]:
tf.constant(11)

<tf.Tensor: shape=(), dtype=int32, numpy=11>

In [9]:
tf.shape(xa)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([2, 3], dtype=int32)>

In [10]:
xa.shape

TensorShape([2, 3])

In [12]:
xa.dtype

tf.float32

###인덱싱

In [13]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [14]:
t[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [16]:
from numpy import arange 
a = arange(16).reshape(2,2,2,2) 
a

array([[[[ 0,  1],
         [ 2,  3]],

        [[ 4,  5],
         [ 6,  7]]],


       [[[ 8,  9],
         [10, 11]],

        [[12, 13],
         [14, 15]]]])

In [17]:
a[..., 0].flatten() 

array([ 0,  2,  4,  6,  8, 10, 12, 14])

In [18]:
a[:,:,:,0].flatten() 

array([ 0,  2,  4,  6,  8, 10, 12, 14])

### 연산

In [19]:
t+10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [20]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [21]:
t @ tf.transpose(t)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

### keras.backend 사용하기

In [22]:
from tensorflow import keras
K = keras.backend
K.square(K.transpose(t)) + 10

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[11., 26.],
       [14., 35.],
       [19., 46.]], dtype=float32)>

### 넘파이 변환

In [24]:
a = np.array([2., 4., 5.])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [25]:
tf.constant(a ,dtype=tf.float32)

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([2., 4., 5.], dtype=float32)>

In [26]:
t

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [27]:
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [1]:
def crash_course(dltr):
    result = 0
    temp = 1

    for i in range(1, dltr):
        temp = temp * i
        for k in range(i):
            result +=temp

    return result

In [2]:
crash_course(4)

23

In [3]:
import numpy as np
a= [[1,2], [2,1]]
b= [[4,1], [2,2]]

np.cross(a,b)

array([-7,  2])

In [4]:
def y(x):
    global a
    a=4
    return 0

def f(a):
    a=3
    print(a)
    return a

a=5
f(a)
print(a)
y(a)
print(a)


3
5
4


In [5]:
np.transpose[[[1,2,3],[4,5,6]]]

TypeError: ignored